In [1]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn

# Importing MNIST data
from tensorflow.examples.tutorials.mnist import input_data
#one_hot encoding true
mnist = input_data.read_data_sets('/tmp/data/', one_hot=True)

/home/suryavamsi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# Training Parameters
learning_rate = 0.003
training_steps = 5000
batch_size = 256
display_step = 400

# Network Parameters
num_input = 28 # as in MNIST data shape of each image is 28*28 pixels
timesteps = 28 # timesteps (t_i in case of an RNN)
num_hidden = 128 # num of hidden layers = num of features
num_classes = 10 # MNIST total classes are 10 as digits are from 0-9.

# tf Graph input
'''
TensorFlow uses a dataflow graph to represent your computation in terms of the dependencies between individual operations.
This leads to a low-level programming model in which you first define the dataflow graph,
then create a TensorFlow session to run parts of the graph across a set of local and remote devices.
'''
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

# Define weights and biases
'''
A variable maintains state in the graph across calls to run(). 
We add a variable to the graph by constructing an instance of the class Variable.
'''

'''
tf.random_normal outputs random values from a normal distribution.
'''
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}
print('Weights')
print(weights)
print('----------\nBiases')
print(biases)

Weights
{'out': <tf.Variable 'Variable:0' shape=(128, 10) dtype=float32_ref>}
----------
Biases
{'out': <tf.Variable 'Variable_1:0' shape=(10,) dtype=float32_ref>}


In [3]:
#Base interface for all RNN Cells in tensorflow.contrib.rnn.RNNCell
# tensorflow.contrib.rnn.BasicLSTMCell is the interface for LSTM cells.

#In this function we define the LSTM cell and preprocess data shape to fit the RNN model.
def RNN(x, weights, biases):

    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a LSTM cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get LSTM cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation function , using rnn inner loop last output (y = Wx+b)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']


In [4]:
# Logit is a function that maps probabilities ([0, 1]) to R ((-inf, inf))
# Probability of 0.5 corresponds to a logit of 0. 
# Negative logit correspond to probabilities less than 0.5, positive to > 0.5.

logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer

# Computes softmax cross entropy between logits and labels
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))

# Here the optimizer chosen is GradientDescent where learning rate is chosen as 0.003

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
# tf.reduce_mean Computes the mean of elements across dimensions of a tensor.
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [5]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= 2.5364, Training Accuracy= 0.102
Step 200, Minibatch Loss= 2.1945, Training Accuracy= 0.227
Step 400, Minibatch Loss= 2.0401, Training Accuracy= 0.297
Step 600, Minibatch Loss= 1.8519, Training Accuracy= 0.367
Step 800, Minibatch Loss= 1.8088, Training Accuracy= 0.414
Step 1000, Minibatch Loss= 1.7126, Training Accuracy= 0.438
Step 1200, Minibatch Loss= 1.6931, Training Accuracy= 0.430
Step 1400, Minibatch Loss= 1.4480, Training Accuracy= 0.539
Step 1600, Minibatch Loss= 1.4345, Training Accuracy= 0.539
Step 1800, Minibatch Loss= 1.4240, Training Accuracy= 0.539
Step 2000, Minibatch Loss= 1.2981, Training Accuracy= 0.555
Step 2200, Minibatch Loss= 1.1102, Training Accuracy= 0.695
Step 2400, Minibatch Loss= 1.0199, Training Accuracy= 0.672
Step 2600, Minibatch Loss= 1.0859, Training Accuracy= 0.656
Step 2800, Minibatch Loss= 1.1325, Training Accuracy= 0.672
Step 3000, Minibatch Loss= 1.0821, Training Accuracy= 0.680
Step 3200, Minibatch Loss= 1.0888, Training Acc